## 模型参数的访问、初始化和共享

在$3.3$节（线性回归的简洁实现）中，我们通过$init$模块来初始化模型的参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何**访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数**。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们从$nn$中导入了$init$模块，它包含了多种模型初始化方法。

In [1]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


### 访问模型参数
我们可以通过$Module$类的$parameters()$或者$named\_parameters$方法来访问所有参数（以迭代器的形式返回），后者除了返回参数$Tensor$外还会返回其名字

In [2]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


对于使用$Sequential$类构造的神经网络，我们可以通过方括号$[]$来访问网络的任一层

注意：如果一个tensor的类型是它的子类Parameter，那么它会自动被添加到模型的参数列表里

In [3]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [4]:
class MyModule(nn.Module):
    def __init__(self, **kwargs):
        super(MyModule, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModule()
for name, param in n.named_parameters():
    print(name)

weight1


上面的代码中$weight1$在参数列表中但是$weight2$却没在参数列表中。

因为$Parameter$是$Tensor$，即$Tensor$拥有的属性它都有，比如可以根据$data$来访问参数数值，用grad来访问参数梯度

In [5]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.2426, -0.4644,  0.0954, -0.4271],
        [ 0.4759,  0.3558, -0.3363,  0.1563],
        [-0.0905, -0.0861, -0.0800, -0.3816]])
None
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.0867, 0.0573, 0.1944],
        [0.0000, 0.0000, 0.0000, 0.0000]])


### 初始化模型参数
$PyTorch$的$init$模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为$0$、标准差为$0.01$的正态分布随机数，并依然将偏差参数清零。不同类型的$layer$具体采样的哪一种初始化方法的可参考源代码）

In [6]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0236,  0.0210,  0.0021,  0.0057],
        [-0.0176, -0.0099, -0.0043, -0.0089],
        [ 0.0147,  0.0020,  0.0082, -0.0009]])
2.weight tensor([[ 4.2154e-05, -4.6273e-03, -1.4081e-02]])


常数初始化偏置参数

In [7]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 自定义初始化方法
有时候我们需要的初始化方法并没有在$init$模块中提供。这时，可以实现一个初始化方法，从而能够像使用其他初始化方法那样使用它。在这之前我们先来看看$PyTorch$是怎么实现这些初始化方法的，例如$torch.nn.init.normal\_$

In [8]:
def normal_(tensor, mead=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)

可见初始化就是对$tensor$进行一个$nograd$的$inplace$

如下实现一个自己的初始化方法，一半的数概率为$0$，另一半在绝对值大于$5$的区间中

In [9]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs()>=5).float()
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-5.2095, -0.0000,  5.1922, -0.0000],
        [-9.6800,  0.0000,  7.6279,  5.5982],
        [ 0.0000,  6.9546, -0.0000,  9.5480]])
2.weight tensor([[-8.2664, -6.7302,  0.0000]])


还可以直接改变参数的$data$达到目的

In [10]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 共享模型参数
$Module$类的$forward$函数里多次调用**同一个层**。此外，如果我们传入$Sequential$的模块是**同一个$Module$实例**的话参数也是共享的。

In [11]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [12]:
print(id(net[0].weight)==id(net[1].weight))

True


因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:

In [13]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
